In [1]:
cd /content/drive/MyDrive/pyimagesearch

/content/drive/Othercomputers/My Laptop/pyimagesearch


In [2]:
mkdir Chapter18

mkdir: cannot create directory ‘Chapter18’: File exists


In [3]:
cd Chapter18

/content/drive/Othercomputers/My Laptop/pyimagesearch/Chapter18


In [4]:
mkdir weight/best

In [5]:
!nvidia-smi

Fri Jul  1 18:04:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Import the necessary packages

In [6]:
import sys
sys.path.append("..")
from sklearn.preprocessing import LabelBinarizer
from nn.conv.minivggnet import MiniVGGNet
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import os

#### Construct the dictionary instead of argument parse

In [7]:
args = {
    'weights': 'weight/best/cifar10_best_weights.hdf5'
}

#### Load the training and testing data, then scale it into the range [0, 1]

In [8]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0

[INFO] loading CIFAR-10 data...
170508288/170498071 [==============================] - 13s 0us/step


#### Convert the label names for the CIFAR-10 dataset

In [9]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

### Initialize the SGD optimizer along with the MiniVGGNet architecture

In [10]:
print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay = 0.01/40, momentum = 0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt,
              metrics=['accuracy'])

[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### Construct the callback to save only the *best* model to disk based on the validation loss

In [11]:
checkpoint = ModelCheckpoint(args['weights'], monitor='val_loss', mode='min',
                             save_best_only=True, verbose=1)
callbacks = [checkpoint]

#### Train the network

In [12]:
# train the network
print('[INFO] training network')
model.fit(trainX, trainY, validation_data=(testX, testY),
          batch_size=64, epochs=40, callbacks = callbacks, verbose=2)


782/782 - 6s - loss: 0.2499 - accuracy: 0.9093 - val_loss: 0.5527 - val_accuracy: 0.8243 - 6s/epoch - 8ms/step
